In [3]:
# imports and settings
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# features/ csv -> dataframe
feat_edu_raw = pd.read_csv("./birth_csv/feat_edu.csv")
df_edu = pd.DataFrame(feat_edu_raw)
feat_hou_raw = pd.read_csv("./birth_csv/feat_hou.csv")
df_hou = pd.DataFrame(feat_hou_raw)
feat_mar_raw = pd.read_csv("./birth_csv/feat_mar.csv")
df_mar = pd.DataFrame(feat_mar_raw)
feat_preg_raw = pd.read_csv("./birth_csv/feat_preg.csv")
df_preg = pd.DataFrame(feat_preg_raw)
feat_sal_raw = pd.read_csv("./birth_csv/feat_sal.csv")
df_sal = pd.DataFrame(feat_sal_raw)

# target/ csv -> dataframe
target_birth_raw = pd.read_csv("./birth_csv/target_birth.csv")
df_birth = pd.DataFrame(target_birth_raw)

# 'ADD_UP'이 포함된 데이터 제거
df_birth = df_birth[~df_birth['AREA'].str.contains('ADD_UP', na=False)]

# 고유한 지역 리스트 추출
areas = df_birth['AREA'].unique()

# feature dict / target dict
features_dict = {}
target_dict = {}

In [4]:
# 각 지역별 feature와 target 데이터 생성
for area in areas:
    area_target = df_birth[df_birth['AREA'] == area].drop(columns='AREA').transpose()
    area_target.columns = ['TARGET']

    # 각 feature 데이터 생성
    area_feat_edu = df_edu[df_edu['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_edu.columns = ['EDU']
    
    area_feat_hou = df_hou[df_hou['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_hou.columns = ['HOU']
    
    area_feat_mar = df_mar[df_mar['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_mar.columns = ['MAR']
    
    area_feat_preg = df_preg[df_preg['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_preg.columns = ['PREG']
    
    area_feat_sal = df_sal[df_sal['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_sal.columns = ['SAL']

    # 지역별 feature 데이터 결합
    area_features = pd.concat([area_feat_edu, area_feat_hou, area_feat_mar, area_feat_preg, area_feat_sal], axis=1)

    # 'ADD_UP'이 포함된 features 데이터 제거
    area_features = area_features[~area_features.index.str.contains('ADD_UP', na=False)]

    # dictionary에 feature와 target 저장
    features_dict[area] = area_features
    target_dict[area] = area_target

In [ ]:
# 각 지역에 대해 학습 및 평가 진행
for area in features_dict.keys():
    print(f"\n{area} 데이터에 대한 학습 시작")

    # 해당 지역의 feature와 target을 가져오기
    X_area = features_dict[area]
    y_area = target_dict[area]

    # 데이터 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_area)

    # train_test_split 80% : 20%
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_area, test_size=0.2, random_state=42)

    # Ridge 회귀 모델 학습
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)

    # 예측
    y_train_pred = ridge_model.predict(X_train)
    y_test_pred = ridge_model.predict(X_test)

    # MSE (Mean Squared Error) for Test Data
    mse = mean_squared_error(y_test, y_test_pred)
    print(f"{area} - 평균 제곱 오차 (MSE) for Test Data: {mse}")

    # R² for Test Data
    r2_test = r2_score(y_test, y_test_pred)
    print(f"{area} - 결정 계수 (R²) for Test Data: {r2_test}")

    # MAE (Mean Absolute Error) for Test Data
    mae = mean_absolute_error(y_test, y_test_pred)
    print(f"{area} - 평균 절대 오차 (MAE) for Test Data: {mae}")

    # R² for Train Data
    r2_train = r2_score(y_train, y_train_pred)
    print(f"{area} - 결정 계수 (R²) for Train Data: {r2_train}")
    print("-" * 50)


SEOUL 데이터에 대한 학습 시작
SEOUL - 평균 제곱 오차 (MSE) for Test Data: 9594794.091047669
SEOUL - 결정 계수 (R²) for Test Data: 0.9827261721457011
SEOUL - 평균 절대 오차 (MAE) for Test Data: 2348.024771272074
SEOUL - 결정 계수 (R²) for Train Data: 0.9656754962724602
--------------------------------------------------

BUSAN 데이터에 대한 학습 시작
BUSAN - 평균 제곱 오차 (MSE) for Test Data: 1377669.4600730832
BUSAN - 결정 계수 (R²) for Test Data: 0.961992155843876
BUSAN - 평균 절대 오차 (MAE) for Test Data: 1043.2840150099923
BUSAN - 결정 계수 (R²) for Train Data: 0.9492282273294245
--------------------------------------------------

DAEGU 데이터에 대한 학습 시작
DAEGU - 평균 제곱 오차 (MSE) for Test Data: 785124.3825693919
DAEGU - 결정 계수 (R²) for Test Data: 0.9677643165337767
DAEGU - 평균 절대 오차 (MAE) for Test Data: 644.7430348707858
DAEGU - 결정 계수 (R²) for Train Data: 0.9625408181068043
--------------------------------------------------

INCHEON 데이터에 대한 학습 시작
INCHEON - 평균 제곱 오차 (MSE) for Test Data: 1612281.658364999
INCHEON - 결정 계수 (R²) for Test Data: 0.9433021